In [1]:
from bs4 import BeautifulSoup
import requests
import pprint
from datetime import datetime

import json

import pandas as pd

from datetime import datetime


In [2]:
team_abbreviations = {'Atlanta Hawks':'ATL', 
                      'Brooklyn Nets':'BKN',
                      'Boston Celtics':'BOS',
                      'Charlotte Hornets': 'CHA',
                      'Chicago Bulls': 'CHI',
                      'Cleveland Cavaliers': 'CLE',
                      'Dallas Mavericks': 'DAL',
                      'Denver Nuggets': 'DEN',
                      'Detroit Pistons': 'DET',
                      'Golden State Warriors': 'GSW',
                      'Houston Rockets':'HOU',
                      'Indiana Pacers': 'IND',
                      'Los Angeles Clippers':'LAC',
                      'Los Angeles Lakers': 'LAL',
                      'Memphis Grizzlies': 'MEM',
                      'Miami Heat': 'MIA',
                      'Milwaukee Bucks':'MIL',
                      'Minnesota Timberwolves':'MIN',
                      'New Orleans Pelicans': 'NOP',
                      'New York Knicks': 'NYK',
                      'Oklahoma City Thunder': 'OKC',
                      'Orlando Magic': 'ORL',
                      'Philadelphia 76ers': 'PHI',
                      'Phoenix Suns': 'PHX',
                      'Portland Trail Blazers': 'POR',
                      'Sacramento Kings': 'SAC',
                      'San Antonio Spurs': 'SAS',
                      'Toronto Raptors': 'TOR',
                      'Utah Jazz' : 'UTA',
                      'Washington Wizards':'WAS'}

city_abbreviations = {'Atlanta':'Atlanta Hawks', 
                      'Brooklyn':'Brooklyn Nets',
                      'Boston':'Boston Celtics',
                      'Charlotte':'Charlotte Hornets',
                      'Chicago':'Chicago Bulls',
                      'Cleveland':'Cleveland Cavaliers',
                      'Dallas':'Dallas Mavericks',
                      'Denver':'Denver Nuggets',
                      'Detroit':'Detroit Pistons',
                      'Golden State':'Golden State Warriors',
                      'Houston':'Houston Rockets',
                      'Indiana':'Indiana Pacers',
                      'LA Clippers':'Los Angeles Clippers',
                      'LA Lakers':'Los Angeles Lakers',
                      'Memphis':'Memphis Grizzlies',
                      'Miami':'Miami Heat',
                      'Milwaukee':'Milwaukee Bucks',
                      'Minnesota':'Minnesota Timberwolves',
                      'New Orleans':'New Orleans Pelicans',
                      'New York':'New York Knicks',
                      'Oklahoma City':'Oklahoma City Thunder',
                      'Orlando': 'Orlando Magic',
                      'Philadelphia':'Philadelphia 76ers',
                      'Phoenix':'Phoenix Suns',
                      'Portland':'Portland Trail Blazers',
                      'Sacramento':'Sacramento Kings',
                      'San Antonio':'San Antonio Spurs',
                      'Toronto':'Toronto Raptors',
                      'Utah':'Utah Jazz',
                      'Washington':'Washington Wizards'}
def get_team_name(bad_str, full_team):
    if full_team:
        return team_abbreviations[bad_str]
    else:
        full_team_name = city_abbreviations[bad_str]
        return get_team_name(full_team_name, True)
print('done')

done


In [5]:
'''
Scrapes data from oddshark and adds it jsonNEWW
'''


columns = ['Win %','avg margin of victory','home or away']

all_data = {}


for year in [2022]:#range(2018, 2022): #from 2018 to 2021
    print('\n'*2)
    print(year)
    for j in range(30):
        print(j)
        URL = 'https://www.oddsshark.com/stats/gamelog/basketball/nba/207'+str(22+j)+'/'+str(year)
        page = requests.get(URL)

        soup = BeautifulSoup(page.content,'html.parser')

        #pprint.pprint(soup)
        title_text = soup.find(id = 'page-title').text.split()
        dash_index = title_text.index('-')
        team = get_team_name(" ".join(title_text[:dash_index]), True)

        print(team)

        results = soup.find(id='content')
        
        if results is None:
            continue
        games = results.find_all('tr')


        
        total_data = {'team':[], 'date':[],'wins':[],'losses':[], 'won by':[],'home or away':[],'spread':[],'won against spread':[],'opponent':[]}
        for game in games:
            data = game.find_all('td')
            new_data = ['Date','Opponent','Game','Result','Score','ATS','Spread']
            storage = {}
            if data:
                for i in range(len(new_data)):
                    storage[new_data[i]] = data[i].text
                
                
                if len(storage['Result']):
                    
                    date_str = storage['Date']
                    split = date_str.split(' ')
                    month_map = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
                    month_num = month_map.index(split[0])+1
                    date_num = int(split[1][:-1])
                    year_num = int(split[2])

                    date = datetime(year_num, month_num, date_num)
                    
                    total_data['date'].append(date)

                    game_opp = storage['Opponent'].split()
                    if '@' in game_opp[0]:
                        home = 0
                    else: home = 1

                    opp_city = " ".join(game_opp[1:])
                    opponent = get_team_name(opp_city, False)

                    spread = eval(storage['Spread'])
                    
                    
                    total_data['team'].append(team)

                    if str(storage['ATS'])== 'W':
                        ATS = 1
                    else: ATS = 0
                    total_data['won against spread'].append(ATS)

                   
                    total_data['home or away'].append(home)
                    
                    total_data['opponent'].append(opponent)
                        
                    won_by = eval(storage['Score'])
                    total_data['won by'].append(won_by)

                    result = storage['Result']
                    try:
                        num_wins = total_data['wins'][-1]
                        num_loss = total_data['losses'][-1]
                    except:
                        num_wins = 0
                        num_loss = 0

                    if result == 'W':
                        total_data['wins'].append(num_wins+1)
                        total_data['losses'].append(num_loss)
                    else:
                        total_data['wins'].append(num_wins)
                        total_data['losses'].append(num_loss+1)

                    
                    total_data['spread'].append(spread)
                
                else:
                    break

            


        formatted_data = {}    
        win_percentage = []
        win_percentage_5 = []
        avg_margin = []
        avg_margin_5 = []
        num_games = len(total_data['wins'])
        for i in range(num_games):
            wins = total_data['wins'][i]
            losses = total_data['losses'][i]

            win_percentage.append(float(wins)/(wins+losses))

            if i<5:
                wins_5 = 0
                losses_5 = 0
                avg_margin_5games = 0
            else:
                wins_5 = total_data['wins'][i-5]
                loss_5 = total_data['losses'][i-5]
                avg_margin_5games = avg_margin[i-5]
            
            last_5_games_wins =  wins-wins_5
            last_5_games_losses = losses-losses_5
            last_5_games_win_percent =float(last_5_games_wins)/(last_5_games_wins+last_5_games_losses)
            win_percentage_5.append(last_5_games_win_percent)

            margin = sum(total_data['won by'][:i+1])/float(i+1)
            avg_margin.append(margin)

            avg_margin_5.append(margin-avg_margin_5games)


        formatted_data['win_percentage_5'] = win_percentage_5
        formatted_data['win_percentage'] = win_percentage
        formatted_data['avg_margin_5'] = avg_margin_5
        formatted_data['avg_margin'] = avg_margin
        formatted_data['home_or_away'] = total_data['home or away']
        formatted_data['spread'] = total_data['spread']
        formatted_data['beat_spread'] = total_data['won against spread']
        formatted_data['opponent'] = total_data['opponent']
        formatted_data['date'] = total_data['date']
        all_data[team] = formatted_data

    def my_converter(o):
        if isinstance(o, datetime):
            return o.__str__()
    with open("scraped-team-data/nba_team_"+str(year)+".json", "w") as outfile:  
        json.dump(all_data, outfile,default = my_converter, indent=4) 




2022
0


AttributeError: 'NoneType' object has no attribute 'text'

In [37]:

import pandas as pd


data_frames = []

for year in range(2018, 2022): 
    print(year)
    data = None
    with open("scraped-team-data/nba_team_"+str(year)+".json", "r") as f:  
        data = json.load(f)

    new_data = {}

    mini_data_frames = []

    for team, values in data.items():
        
        mini_data = values.copy()
        
        num_games = len(values['win_percentage'])

        opp_win_percentage_5  = []
        opp_win_percentage = []
        opp_avg_margin_5 = []
        opp_avg_margin = []
        

        for i in range(num_games):
            
            date = mini_data['date'][i]
            opponent = values['opponent'][i]
            opp_values = data[opponent]
            opp_index = opp_values['date'].index(date)
            opp_win_percentage_5.append(opp_values['win_percentage_5'][opp_index])
            opp_win_percentage.append(opp_values['win_percentage'][opp_index])
            opp_avg_margin_5.append(opp_values['avg_margin_5'][opp_index])
            opp_avg_margin.append(opp_values['avg_margin'][opp_index])

        mini_data['opp_win_percentage_5'] = opp_win_percentage_5
        mini_data['opp_win_percentage'] = opp_win_percentage
        mini_data['opp_avg_margin_5'] = opp_avg_margin_5
        mini_data['opp_avg_margin'] = opp_avg_margin
        

        mini_data.pop('opponent')

        mini_data_frames.append(pd.DataFrame.from_dict(mini_data))
    
    
    data_frames.append(pd.concat(mini_data_frames))


all_data = pd.concat(data_frames)

with open('all_data.csv', 'w') as train:
    all_data.to_csv(train)





2018
2019
2020
2021


In [38]:

training_data = pd.read_csv('all_data.csv')
print(training_data.shape[0])

9876
